# Functions, State, and Decorators


A function is a set of statements that take inputs, do some specific computation and produces output. 

- Functions can be built-in, imported or user defined
- A function can be called from other functions
- Can return data

You have already used functions. Some of them belong to objects of specific types, such as `dict_var.get`, and are called **methods**, while some of them have been built-in to the python system, such as `print`. Some other built-ins you have seen are `list` and `dict`, and these are **constructor** functions that create objects of given types.

Here is another example of a built-in function

In [1]:
#Built-in functions

var1 = -15
abs(var1)

## Defining your own functions

- You can take a number of arguments as input with varying data types. 
- The arguemnts can optionally have default values which are used in case of no values passed to the function.

In [4]:
# User-defined function with one argument

def function_name1(var1):
    var2 = 10
    x = var1 + var2
    return x

var1 = 6
var2 = function_name1(var1)
print (var2)

In [5]:
#User defined function with 2 arguments, where the second one is a list with default values.
#A function can also 

def function_name2(var2, l1=[1,2,3,4]):
    l2 = []
    for i in l1:
        l2.append(var2+i)
    return l2

var2 = 6
l = function_name2(var2)
print ("This is the list that has been returned ", l)

In [6]:
function_name2(var2, [0,0,0,0])

In [7]:
function_name2(var2, l1=[0,0,0,0])

Functions may also be defined using the so-called **lambda** or anonymous function, in which functions are then assigned to variables:

In [9]:
square = lambda x: x*x
affine = lambda a,b,x: a+b*x

These are particularly useful for math where most functions are algebraic 1-liners

In [10]:
square(5)

In [11]:
affine(1,2,5)

## The scope of variables in functions

- Scope of a variable is the portion of a program where the variable is recognized. Parameters and variables defined inside a function is not visible from outside. Hence, they have a local scope.

- Lifetime of a variable is the period throughout which the variable exits in the memory. The lifetime of variables inside a function is as long as the function executes.

- They are destroyed once we return from the function. Hence, a function does not remember the value of a variable from its previous calls.

The scope of this jupyter notebook, or in a python file, is the **global scope**. The scope defined inside of a function definition is the **local** scope.

Here is an example to illustrate the scope of a variable inside a function.

In [8]:
def my_func():
    x = 10
    print("Value inside function:",x)

x = 20
my_func()
print("Value outside function:",x)

In this way, a variable defined locally can shadow a global. Here the value `x` inside `my_func`comes from the local definition (10), not the global one (20)

## Functions are objects

Often you will hear it said that *functions are first class objects*. This means that functions can act as objects, and thus be represented as variables. For example:

In [12]:
square = lambda x: x*x

The further meaning of this is that you can return functions from functions just as you return variables, and pass functions into functions, just as you would pass variables. This means that you can achieve very general functionality easily, for examplethe `map` in map-reduce frameworks

In [13]:
def mapit(listy, func):
    return [func(e) for e in listy]
mapit(range(5), square)

But python goes further! You can define functions inside of functions and return your defined functions..this is the other side of the coin of taking functions of arguments..you can return them as well. This further expands the menu of things you can do. For example:

In [14]:
def soa(f): # sum anything
    def h(x, y):
        return f(x) + f(y)
    return h

Gere we are writung a function `soa` that takes a function `f` as an argument, and returns a function `h`, which when executed takes two imputs, puss them through `f`, and then sums them. So:

In [16]:
sum_of_squares = soa(square)
type(sum_of_squares)

In [17]:
sum_of_squares(3,4)

## Capturing state and creating function decorators

One of the key things enabled by this functionality is the capturing of state.

In [18]:
def soaplusbias(f, bias): # sum anything
    def h(x, y):
        return f(x) + f(y) + bias
    return h
sosplusbias = soaplusbias(lambda x: x*x, 5)
sosplusbias(3, 4) 

Notice what has happened here. In the outer function we added another argument. When we call `soaplusbias`, we define the internal function. But in this definition we now capture the calue of the bias from the execution of the outer function. Thus that value of 5 is used when we execute `sosplusbias(3, 4)`, having been captured when it was defined. This idea is called a `closure`, in the sense that this bias has been closed/captured into the definition.

Its used everywhere, from defining callbacks in graphical user interfaces when responding to mouse clicks, to defining callbacks used to, for example, display the loss in deep learning systems such as `keras`. You will be writing them soon.

The paradigm of an outer function wunning to define an inner function is so common, that Python has a special syntax for it. This is the syntax of decorators.

In [23]:
def check_posint(f):
    def checker(n):
        if n > 0:
            return f(n)
        elif n == 0:
            return 1
        else:
            raise ValueError("Not a positive int")
    return checker

@check_posint
def factorial(n):
    return n*factorial(n-1)
    
print(factorial(4)) # returns 24

Here `check_posint` is a general function which checks an integer is positibe before passing through to function f. We **decorate** the factorial function with it, which essentially "captures" the function f into the `checker`. The syntax:

```python
@check_posint
def factorial(n):
    return n*factorial(n-1)
```

is eaquivalent to

```python
factorial = check_posint(factorial)
```

with some additional metadata changes on the functional object which are not relevant for us now. So we have replced the `factorial` function by the `checker` function, which captures the factorial function inside it and calls it when called.



In [24]:
%timeit factorial(15)

The `%timeit` here is called a **jupyter cell magic**, and implements a timing loop for us, running 7 runs with 100000 calls, and averaging)

## Refactoring

We earlier saw closures in which we captured a variable from outside our scope. Our example had a preset bias that we put in. Let us now see how we can use state as captured in a cache to spped up our computation of `factorial`

In [26]:
global_cache1 = {}
def check_posint_global_cache(f):
        def checker(n):
            if n > 0:
                if n in global_cache1:
                    return global_cache1[n]
                else:
                    val = f(n)
                    global_cache1[n] = val
                    return val
            elif n == 0:
                return 1
            else:
                raise ValueError("Not a positive int")
        return checker

In [27]:
@check_posint_global_cache
def factorial2(n):
    return n*factorial2(n-1)

In [28]:
%timeit factorial2(15)

Look at the speedup we get from using a cache! The first time we run, stuff is put into the cache, and then just obtained from there! So now even getting the factorial of a larger number, say 20, should be faster, as  we have caching upto 15.

In [35]:
%time factorial2(20)

In [36]:
%time factorial2(20)

The original call is much slower...(we have used another magic called %time to run the code just once).

In [38]:
%time factorial(20)

In [39]:
%timeit factorial2(20)

At this point you say but this is ugly..you are encoding globals into your function, which means you cannot use this function anywhere else.

There is a price in complexity though: we must now have two wrapper functions because we need to capture the cache into a closure, while keeping the inner wrapper function looking like before.

In [30]:
def check_posint_and_cache(cache):
    def check_posint(f):
        def checker(n):
            if n > 0:
                if n in cache:
                    return cache[n]
                else:
                    val = f(n)
                    cache[n] = val
                    return val
            elif n == 0:
                return 1
            else:
                raise ValueError("Not a positive int")
        return checker
    return check_posint

In [31]:
global_cache2 = {}
def factorial3(n):
    return n*factorial3(n-1)
factorial3 = check_posint_and_cache(global_cache2)(factorial3)

In [32]:
%timeit factorial3(15)

Python provides simple syntax for this.
This syntax will call all functions down to but not including the innermost function, which is replaced by its (now multiple called wrappers). Thus calling the decorator on a function returns a function that when called (so 2 calls happen at define time) creates a function to replace our factorial function.

In [33]:
global_cache3 = {}


@check_posint_and_cache(global_cache3)
def factorial4(n):
    return n*factorial4(n-1)
    


In [34]:
%timeit factorial4(15)

Our decorator can now be put into a library and used for multiple things! This process of developing functionality and then changing it to be more general is called refactoring, and is something you should always do after writing your initial code. We'll have more to talk about it soon.